In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年12月20日(土)～2026年2月22日(日). 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都). CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ). 日比谷駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 京橋駅(東京都). 開催中 2026年1月2日(金)～3月22日(日). 二重橋前駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). * 歴史リアル謎解きゲーム「謎の城」in 日本橋「発明家／人斬り-平賀源内-」. 開催中 2025年9月1日(月)～2026年3月1日(日). 日本橋駅(東京都), 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 三越前駅(東京都). * MIDTOWN YAESU CHRISTMAS 2025 (ミッドタウン八重洲クリスマス2025). 終了間近 2025年11月13日(木)～2026年2月15日(日). 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都). * 丸の内から御食国「敦賀・若狭」へ 美し物-うましもの-グルメフェア. 開催中 2026年2月2日(月)～21日(土). 二重橋前駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 日比谷駅(東京都). 終了間近 2025年11月13日(木)～2026年2月15日(日). 二重橋前〈丸の内〉駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). * Masking Tape Jamboree in KITTE2026. 終了間近 2026年2月11日(水)～13日(金). 東京駅(東京都), 二重橋前駅(東京都), 有楽町駅(東京都), 京橋駅(東京都), 銀座一丁目駅(東京都). 東京駅(東京都), 二重橋前駅

In [8]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [4]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [5]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [6]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [9]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の広さを比較すると、沖縄県の方が広いです。

- **東京都**の面積は約2,194平方キロメートルです。
- **沖縄県**の面積は約2,281平方キロメートルです。

したがって、沖縄県の方が東京都よりも少し広いです。


In [10]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内に東京駅で行われるイベントに関する情報をいくつかお知らせします。

1. **50代・60代・クリスマス特別・笑顔あふれる食事会**
   - 日時: 未定
   - 内容: 「散策・街歩き」や「自然景観」「国内旅行」「温泉」などに関心がある方々のための食事会。景色や食べ物を楽しむ機会です。
   - 詳細情報: [詳細はこちら](https://www.ya-7.com/event_info.php?EventNum=24461&ya7_domain_name=ya7)

2. **東京駅周辺 明日のおすすめのイベント**
   - 内容: 東京駅周辺で開催されるイベントの情報をまとめて紹介。
   - 詳細情報: [詳細はこちら](https://www.enjoytokyo.jp/event/list/sta200101/its05/)

3. **東京駅のイベントとニュース**
   - 内容: 各種イベントやキャンペーン情報を豊富に掲載。
   - 詳細情報: [詳細はこちら](http://www.tokyostationcity.com/news/)

4. **イベント情報のまとめ**
   - 内容: Tokyo Stationの各種イベント情報や、買い物などの情報をまとめたページ。
   - 詳細情報: [ここから検索](https://t.livepocket.jp/event/search?venue=%7B%E6%9D%B1%E4%BA%AC%E9%A7%85%7D)

これらのリンクを参考に、興味のあるイベントをチェックしてみてください。


In [12]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:最近のPython開発環境のトレンドを教えて'

最近のPython開発環境に関するトレンドは次のような点が挙げられます。

1. **新規参入者の増加**:
   新しい開発者の増加に伴い、初心者向けの教育コンテンツやリソースが豊富に提供されています。特に、Pythonはデータサイエンスや機械学習の分野での人気が高まり、新規参入者が多くなっています。

2. **データサイエンスとAIの活用**:
   Pythonは、データ分析や人工知能、機械学習に特化したライブラリ（例えばPandasやNumPy、Scikit-learnなど）を豊富に備えており、これらの技術の普及とともに使用が広まっています。

3. **開発環境の多様化**:
   開発環境としては、Visual Studio CodeやPyCharmが人気ですが、Jupyter Notebookのようなインタラクティブな環境もデータサイエンスコミュニティで広く使われています。

4. **モダンな開発手法の導入**:
   FastAPIやFlaskなどの軽量なフレームワークを用いたAPI開発が増加し、モダンなアーキテクチャ（例えばマイクロサービスなど）との統合が進んでいます。

5. **仮想環境とパッケージ管理の進化**:
   `venv`や`pip`に加え、`poetry`や`pipenv`といった新しいツールが登場し、パッケージ管理や環境構築がより簡単に行えるようになっています。

6. **バージョン管理の重要性**:
   バージョン管理システム（Git等）の利用がますます重要視され、チーム開発においてはこれらのツールが必須となっています。

7. **コンテナ技術の利用**:
   DockerやKubernetesを活用した開発、デプロイメントの流れが浸透し、特にスケーラブルなアプリケーションを作成するための基盤として重要視されています。

8. **上述のトレンドが示すように、Pythonは今後も多くの分野での活用が期待されています**。特に、AIやデータサイエンスの分野においては、その重要性が増す一方で、人材の需要も高まっています。

これらの情報は、Pythonコミュニティの動向と最新の技術、実践に基づいています。詳細を知りたい場合は、関連するブログやリサーチを参照すると良いでしょう。

---ご利用ありがとうございました！--